In [18]:
#Title:  The Timing of Pollen Deposition in Crater Lake National Park based on Climate Conditions

#Objective: To determine the timing of pollen deposition as it correlates to climate conditions

#Background: Crater Lake is the second deepest lake in the world and a uniquely isolated environment. There is little nutrient input into the lake because it is is the caldera of a dormant volcano. As a result, pollen is one of the few nutrient sources from mirobes and zooplanketon which ultimately feed the stocked fish population.

#Datasets to Use: Landsat-8 and Sentinel-2 MSI


In [19]:
#First a map and calling in our desired location. Code is from Dr. Torres, Lab 2, 2020

In [20]:
# Initializing display and earthengine
from IPython.display import Image
%matplotlib inline

import ee
ee.Initialize()

# importing ipygee for dynamic mapping
from ipygee import *

Map = Map() # from ipygee
Map.show()

Map(center=[0, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text'…

In [21]:
#Selecting the area of interest...In this case it is Crater Lake National Park
latitude =  42.941735
longitude =-122.108748
CL_Point = ee.Geometry.Point([longitude, latitude]); #Location of Crater Lake

In [22]:
#Calling in Sentinel-2 MSI Data for use in the project. Again, this code is developed by Dr. Torres and is based on previous labs, this is from lab 2.
#10m resolution for rbg
sent2=ee.ImageCollection("COPERNICUS/S2_SR")

# // Filter sent2 imagery by location, date and cloudiness.
sent2.Image = ee.Image(sent2
    .filterBounds(CL_Point) 
    .filterDate('2019-03-01', '2019-06-30') #data only available from 2017 onwards
    .sort('CLOUDY_PIXEL_PERCENTAGE')
    .first());

# Print the image band names to the console.
band_names = sent2.Image.bandNames()
print(band_names.getInfo())

# // Display the vis of sentinel 2.
sent2Bands= ['B4','B3','B2']#'B4','B3','B2

Map.addLayer(sent2.Image, {'bands':sent2Bands,'min': 0, 'max': 1500}, name='Sent-2 MSI Image')

bounds = CL_Point.buffer(70000)
Map.centerObject(bounds)

['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12', 'AOT', 'WVP', 'SCL', 'TCI_R', 'TCI_G', 'TCI_B', 'MSK_CLDPRB', 'MSK_SNWPRB', 'QA10', 'QA20', 'QA60']


In [23]:
#Landsat 
Landsat=ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")

# Filter landsat imagery by location, date and cloudiness.
Landsat_Image = ee.Image(Landsat
    .filterBounds(CL_Point) 
    .filterDate('2015-05-15', '2015-06-30') #data only available from 2013 onwards
    .sort('CLOUD_COVER')
    .first());

# Display the IR of landsat
LandsatBands= ['B5','B4','B3'] #chose IR just cause the pollen contrasts more

Map.addLayer(Landsat_Image, {'bands':LandsatBands,'min': 0, 'max': 1500}, name='Landsat - 8')

bounds = CL_Point.buffer(5000)
Map.centerObject(bounds)

In [24]:
#classification for pollen

polygon_1 = ee.Geometry.Polygon([
                [-122.0824666254484, 42.96477907096206],
                [-122.08242371762178, 42.964182386715855],
                [-122.08394694546685, 42.964213791294135],
                [-122.08418293851327, 42.965564172995016]]);

polygon_2 = ee.Geometry.Polygon([
    [-122.11954470989436, 42.94532104219341],
    [-122.11729215004362, 42.94676607903181],
    [-122.11574753757459, 42.94673466554818],
    [-122.11840770349353, 42.94558806242496]]);

# Create a Feature from the Geometry.
pollen_1= ee.Feature(polygon_1, {'class': 2, 'name': 'pollen_1'});
pollen_2 = ee.Feature(polygon_2, {'class': 2, 'name': 'pollen_2'});
pollen=(pollen_1,pollen_2)
Map.addLayer(pollen_1, {'fill_color':'yellow', 'outline': 1}, name='pollen_1')
Map.addLayer(pollen_2, {'fill_color':'yellow', 'outline': 1}, name='pollen_2')

In [25]:
# classification for Water

polygon = ee.Geometry.Polygon([
        [-122.14472061734979, 42.933086444940585],
        [-122.14369087570375, 42.93072986568092],
        [-122.15051291410879, 42.930478491904516],
        [-122.1475524068764, 42.93233234938799]
                     ]);


# Create a Feature from the Geometry.
water = ee.Feature(polygon, {'class': 0, 'name': 'water'});
Map.addLayer(water, {'fill_color':'blue', 'outline': 1}, name='water')

In [26]:
# classification for Land

polygon = ee.Geometry.Polygon([
        [-122.18212733292346, 42.983843912447654],
        [-122.15947301671059, 42.97354563844106],
        [-122.10017639339559, 43.00422344338795],
        [-122.13862008151442, 43.03384292596175]]);
   

# Create a Feature from the Geometry.
land = ee.Feature(polygon, {'class': 1, 'name': 'land'});
Map.addLayer(land, {'fill_color':'green', 'outline': 1}, name='land')


# classification for cloud

#polygon = ee.Geometry.Polygon([
       # [-122.12261347140866, 42.94813255005339],
      #  [-122.11679946090153, 42.94593361629946],
       # [-122.11546931827628, 42.94784983439976],
       # [-122.1208113426906, 42.94920057524839]]);

# Create a Feature from the Geometry.
#cloud = ee.Feature(polygon, {'class': 1, 'name': 'cloud'});
#Map.addLayer(cloud, {'fill_color':'purple', 'outline': 1}, name='cloud')


In [27]:
#training to recognize pollen in the lake

training = ee.FeatureCollection([water, land, pollen_1,pollen_2])

bands = ['B6', 'B3', 'B2']

classifierTraining = Landsat_Image.select(bands).sampleRegions(
      collection=training, 
      properties=['class'], 
      scale=30
);

#From lab 5, Randomly split the data into 60% for training, and 40% for testing
trainingTesting = classifierTraining.randomColumn('random',111009);

training = trainingTesting.filter(ee.Filter.lt('random', 0.5));

testing = trainingTesting.filter(ee.Filter.gte('random', 0.5));

In [28]:
#random forest 

rfClassification = ee.Classifier.smileRandomForest(numberOfTrees=1, seed=111009).train(
      features= training, 
      classProperty= 'class', 
      inputProperties= bands
    )

# // Perform the RF regression on the landsat image
rfClassificationImage =Landsat_Image.select(bands).classify(rfClassification);
    
# // Visualize the RF regression
Map.addLayer(rfClassificationImage,  {'min': 0, 'max': 2,
                                   'palette':['blue','green', 'yellow']}, 'RF classification');

In [29]:
#classifying 2013 - 2019 

#2013 

# Filter landsat imagery by location, date and cloudiness.
LS2013_Image = ee.Image(Landsat
    .filterBounds(CL_Point) 
    .filterDate('2013-07-16', '2013-07-30') #data only available from 2013 onwards
    .sort('CLOUD_COVER')
    .first());

Map.addLayer(LS2013_Image, {'bands':LandsatBands,'min': 0, 'max': 1500}, name='Landsat - 2013')

# // Perform the RF regression on the landsat image
rfClassificationImage =LS2013_Image.select(bands).classify(rfClassification);
    
# // Visualize the RF regression
Map.addLayer(rfClassificationImage,  {'min': 0, 'max': 2,
                                   'palette':['blue','green', 'yellow']}, 'RF classification - 2013');
###################### 2014
# Filter landsat imagery by location, date and cloudiness.
LS2014_Image = ee.Image(Landsat
    .filterBounds(CL_Point) 
    .filterDate('2014-05-15', '2014-06-30') #data only available from 2013 onwards
    .sort('CLOUD_COVER')
    .first());

Map.addLayer(LS2014_Image, {'bands':LandsatBands,'min': 0, 'max': 1500}, name='Landsat - 2014')

# // Perform the RF regression on the landsat image
rfClassificationImage =LS2014_Image.select(bands).classify(rfClassification);
    
# // Visualize the RF regression
Map.addLayer(rfClassificationImage,  {'min': 0, 'max': 2,
                                   'palette':['blue','green', 'yellow']}, 'RF classification - 2014');
###################### 2015
# Filter landsat imagery by location, date and cloudiness.
LS2015_Image = ee.Image(Landsat
    .filterBounds(CL_Point) 
    .filterDate('2015-05-15', '2015-06-30') #data only available from 2013 onwards
    .sort('CLOUD_COVER')
    .first());

Map.addLayer(LS2015_Image, {'bands':LandsatBands,'min': 0, 'max': 1500}, name='Landsat - 2015')

# // Perform the RF regression on the landsat image
rfClassificationImage2015 =LS2015_Image.select(bands).classify(rfClassification);
    
# // Visualize the RF regression
Map.addLayer(rfClassificationImage2015,  {'min': 0, 'max': 2,
                                   'palette':['blue','green', 'yellow']}, 'RF classification - 2015');

###################### 2016
# Filter landsat imagery by location, date and cloudiness.
LS2016_Image = ee.Image(Landsat
    .filterBounds(CL_Point) 
    .filterDate('2016-05-15', '2016-06-30') #data only available from 2013 onwards
    .sort('CLOUD_COVER')
    .first());

Map.addLayer(LS2016_Image, {'bands':LandsatBands,'min': 0, 'max': 1500}, name='Landsat - 2016')

# // Perform the RF regression on the landsat image
rfClassificationImage =LS2016_Image.select(bands).classify(rfClassification);
    
# // Visualize the RF regression
Map.addLayer(rfClassificationImage,  {'min': 0, 'max': 2,
                                   'palette':['blue','green', 'yellow']}, 'RF classification - 2016');

###################### 2017
# Filter landsat imagery by location, date and cloudiness.
LS2017_Image = ee.Image(Landsat
    .filterBounds(CL_Point) 
    .filterDate('2017-05-15', '2017-06-30') #data only available from 2013 onwards
    .sort('CLOUD_COVER')
    .first());

Map.addLayer(LS2017_Image, {'bands':LandsatBands,'min': 0, 'max': 1500}, name='Landsat - 2017')

# // Perform the RF regression on the landsat image
rfClassificationImage =LS2017_Image.select(bands).classify(rfClassification);
    
# // Visualize the RF regression
Map.addLayer(rfClassificationImage,  {'min': 0, 'max': 2,
                                   'palette':['blue','green', 'yellow']}, 'RF classification - 2017');

###################### 2018
# Filter landsat imagery by location, date and cloudiness.
LS2018_Image = ee.Image(Landsat
    .filterBounds(CL_Point) 
    .filterDate('2018-05-15', '2018-06-30') #data only available from 2013 onwards
    .sort('CLOUD_COVER')
    .first());

Map.addLayer(LS2018_Image, {'bands':LandsatBands,'min': 0, 'max': 1500}, name='Landsat - 2018')

# // Perform the RF regression on the landsat image
rfClassificationImage2018 =LS2018_Image.select(bands).classify(rfClassification);
    
# // Visualize the RF regression
Map.addLayer(rfClassificationImage2018,  {'min': 0, 'max': 2,
                                   'palette':['blue','green', 'yellow']}, 'RF classification - 2018');

###################### 2019
# Filter landsat imagery by location, date and cloudiness.
LS2019_Image = ee.Image(Landsat
    .filterBounds(CL_Point) 
    .filterDate('2019-05-15', '2019-06-30') #data only available from 2013 onwards
    .sort('CLOUD_COVER')
    .first());

Map.addLayer(LS2019_Image, {'bands':LandsatBands,'min': 0, 'max': 1500}, name='Landsat - 2019')

# // Perform the RF regression on the landsat image
rfClassificationImage2019 =LS2019_Image.select(bands).classify(rfClassification);
    
# // Visualize the RF regression
Map.addLayer(rfClassificationImage2019,  {'min': 0, 'max': 2,
                                   'palette':['blue','green', 'yellow']}, 'RF classification - 2019');

In [30]:
# trying to count the number of pollen pixels 

#defining lake area
region = ee.Geometry.Polygon([

[-122.13196812911212, 42.913334540326886],
[-122.13073023812832, 42.91256546972428],
[-122.1297004502894, 42.91240831490808],
[-122.128842293757, 42.912439745903356],
[-122.12789832157134, 42.911936848055085],
[-122.12678271807921, 42.91165396621211],
[-122.12583874589353, 42.91177969163596],
[-122.124766050228, 42.911936848055085],
[-122.12339299977614, 42.912345452869396],
[-122.12236321193724, 42.91256546972428],
[-122.12129051627173, 42.91300550107769],
[-122.12068980669905, 42.91287977815445],
[-122.11991746581985, 42.91262833153854],
[-122.11858732319462, 42.91187398553551],
[-122.11781340361576, 42.91127678840176],
[-122.11699815490998, 42.910428129889034],
[-122.11626872185742, 42.9097366217203],
[-122.11489567140558, 42.90857361321324],
[-122.11480985575231, 42.9081021170265],
[-122.11395169921991, 42.90759918378543],
[-122.11335098964723, 42.90709624644085],
[-122.11232120180831, 42.906907643878704],
[-122.11163467658237, 42.90640470089182],
[-122.11086233570319, 42.90637326681891],
[-122.11004708699741, 42.9067190407395],
[-122.10897439133187, 42.90631039862497],
[-122.10790169566638, 42.90612179365845],
[-122.10704353913394, 42.90574458199423],
[-122.10571339650869, 42.90539880260768],
[-122.10451197736333, 42.905273064168064],
[-122.1030102034316, 42.90502158651943],
[-122.10185169211282, 42.90480154273535],
[-122.10047864166094, 42.90477010784493],
[-122.09940594599544, 42.904581498165825],
[-122.09854778946301, 42.90436145281088],
[-122.09764321437665, 42.90433001769607],
[-122.09648470305788, 42.90442432299244],
[-122.09536909956574, 42.9045500631632],
[-122.09429640390022, 42.90480154273535],
[-122.09322370823469, 42.905555975296544],
[-122.09206519691593, 42.90653043702325],
[-122.09133576386337, 42.90785065091891],
[-122.09056342298419, 42.90854218024635],
[-122.0907350542907, 42.91030240170813],
[-122.08987528898814, 42.90992521562663],
[-122.08923167158882, 42.90976805407808],
[-122.08833060722979, 42.909830918745605],
[-122.08742954287075, 42.90986235105532],
[-122.08648557068507, 42.90995664788826],
[-122.0858419532858, 42.91077388106406],
[-122.08485507327349, 42.91086817650244],
[-122.08356783847488, 42.91099390352924],
[-122.08313876020866, 42.910679585481404],
[-122.08210897236977, 42.910742449219214],
[-122.08073592191789, 42.910742449219214],
[-122.07979194973224, 42.91118249358852],
[-122.07803272884077, 42.91121392520897],
[-122.07691712534863, 42.91181112295183],
[-122.07541535141692, 42.91240831490808],
[-122.07438556357803, 42.91344552928932],
[-122.07373931501257, 42.914639875740974],
[-122.07129356889521, 42.91636849408326],
[-122.07022087322969, 42.9165884965757],
[-122.06863328364473, 42.917625640603255],
[-122.06623044535392, 42.91944845750091],
[-122.06562973578126, 42.91960559436396],
[-122.06502902620856, 42.92098838147874],
[-122.06429959315601, 42.9218683207574],
[-122.06477141386127, 42.923188206111504],
[-122.06562957039367, 42.92419381406861],
[-122.06588701735339, 42.92567077099603],
[-122.06528630778068, 42.926990574886005],
[-122.06387034950221, 42.92743050323224],
[-122.06249729905036, 42.92831035049902],
[-122.0606496839629, 42.92919018519821],
[-122.05944826481749, 42.9301014293163],
[-122.05859010828507, 42.93013285128685],
[-122.05734578131306, 42.93157824460199],
[-122.05631599347417, 42.93157824460199],
[-122.05627308564755, 42.93299218350147],
[-122.05683088739362, 42.933306387737474],
[-122.05644471695403, 42.93478312617409],
[-122.05575867285562, 42.935285837562574],
[-122.05597321198873, 42.940218475360865],
[-122.05665973721464, 42.94163221588378],
[-122.05726044678734, 42.942103455510974],
[-122.05601651073147, 42.94241761325861],
[-122.05666012813076, 42.94496223193338],
[-122.05721792987681, 42.94571617282346],
[-122.05653140465088, 42.94681565340303],
[-122.0550725385458, 42.9472240269024],
[-122.05430019766662, 42.94826065515642],
[-122.05369737564341, 42.94920302932907],
[-122.05266758780452, 42.94929726595281],
[-122.05030765734038, 42.95237558299084],
[-122.04996439472741, 42.95504542716883],
[-122.05069382777997, 42.957149810949026],
[-122.05232505440044, 42.95966241373621],
[-122.05485661617108, 42.96135836261618],
[-122.05691619184888, 42.962394752807526],
[-122.05991973971231, 42.96274021232502],
[-122.06519892846303, 42.96650873559908],
[-122.06665779456814, 42.967293815557255],
[-122.06953261895174, 42.967545039027684],
[-122.07017623635106, 42.96911516247201],
[-122.07210947163374, 42.97071664709546],
[-122.07356833773883, 42.97084225196911],
[-122.07609989950946, 42.97231809002024],
[-122.07442649427125, 42.975552249562625],
[-122.07618609104358, 42.97759313649658],
[-122.07798821976164, 42.97872344472911],
[-122.07961871717325, 42.97888043034034],
[-122.08090595197186, 42.978597855951534],
[-122.08249354155681, 42.977436147598766],
[-122.08395240766194, 42.97756173874906],
[-122.08729921813838, 42.97705937260877],
[-122.08910134685645, 42.97589763518988],
[-122.09077831544532, 42.97630581570291],
[-122.09429675722824, 42.97627441729811],
[-122.09579853115996, 42.97599183093337],
[-122.09691413465211, 42.976243018877284],
[-122.09794392249103, 42.975709243270046],
[-122.09927406511625, 42.97558364833616],
[-122.09983186686233, 42.974296285474544],
[-122.10103328600772, 42.973385695470725],
[-122.10184853471351, 42.97332289566313],
[-122.10339321647184, 42.97310309583154],
[-122.10480917475036, 42.9728832952144],
[-122.10566733128275, 42.97310309583154],
[-122.10764434643566, 42.97536385658468],
[-122.10884576558107, 42.97636861246444],
[-122.11099115691208, 42.97702797458872],
[-122.1128361934568, 42.9774047497711],
[-122.11455250652162, 42.97724776039236],
[-122.11777059351819, 42.976462807486506],
[-122.12051669442192, 42.975646445835174],
[-122.12455003012428, 42.97398229044453],
[-122.12738194668127, 42.972506492328755],
[-122.128883720613, 42.97131326796072],
[-122.1288408127864, 42.97043403519191],
[-122.12884460918318, 42.969994414094174],
[-122.13373610141795, 42.96842431309426],
[-122.1348517049101, 42.968173093214936],
[-122.13669674145478, 42.96848711790377],
[-122.13789816060019, 42.96770205317569],
[-122.14111624759673, 42.96770205317569],
[-122.14600773983152, 42.964687311508015],
[-122.14592192417825, 42.961483986599525],
[-122.14708043549703, 42.96082445782548],
[-122.14991235205402, 42.960981489127235],
[-122.15201666985708, 42.95941115807452],
[-122.15514894120041, 42.95677291168168],
[-122.15733724035806, 42.95388327393874],
[-122.15883952548779, 42.95108773439081],
[-122.1590969724475, 42.94816641694563],
[-122.16008385245979, 42.94694130707598],
[-122.16029839159289, 42.94559051664967],
[-122.16034235940357, 42.94483657422064],
[-122.16231611942811, 42.94357998298621],
[-122.16227321160152, 42.94310875466064],
[-122.16175831768206, 42.942637522728155],
[-122.16188704116192, 42.94219770300361],
[-122.16261647421447, 42.94163221588378],
[-122.16278810552096, 42.9405954760077],
[-122.16386080118649, 42.93723380543148],
[-122.16459010643953, 42.936731109945164],
[-122.16441847513303, 42.93258371559812],
[-122.1634745029474, 42.92966152004592],
[-122.16313124033444, 42.92827892759876],
[-122.16197212008866, 42.92614213277423],
[-122.161028147903, 42.92479088602662],
[-122.15853949395898, 42.92227685972196],
[-122.15708062785389, 42.92076839469545],
[-122.15562025070767, 42.91973130356588],
[-122.15360358285649, 42.918568483662106],
[-122.15304578111041, 42.918317060257024],
[-122.15201599327153, 42.91819134816977],
[-122.15047131151316, 42.917279927866986],
[-122.14939861584763, 42.91652563880085],
[-122.14866918279509, 42.916337065091675],
[-122.14768230278281, 42.91608563258173],
[-122.14592308189134, 42.9157084818933],
[-122.1451936488388, 42.91526846983942],
[-122.14377769056031, 42.91511132191591],
[-122.1417181148825, 42.91357125105846],
[-122.14017343312416, 42.913351237794195],
[-122.13742733222041, 42.91313122374446],
[-122.13583974263545, 42.91281691659664],
[-122.13403761391736, 42.91262833153854],
[-122.13287910259861, 42.91313122374446]]);

#add to map 
CL = ee.Feature(region, {'class': 0, 'name': 'CL'});
Map.addLayer(CL, {'fill_color':'purple', 'outline': 1}, name='CL')

In [31]:
################ 2015 Area ############################

In [32]:
#calculating areage 

# THIS CODE CREATES A BINARY IMAGE OF YOUR INDEX, POSITIVE VALUES ARE 1 NEGATIVE VALUES 0
binary_im =rfClassificationImage2015.eq(2) #class 2 is pollen, any other class is ignored
#Map.addLayer(binary_im)

# // Reduce the region. The region parameter is the Feature geometry. THIS IS THE CODE FOR ADDING THE BINARY IMAGE
meanDictionary2 = binary_im.reduceRegion(
  reducer= ee.Reducer.sum(), #sum of ones from the binary map
  geometry= region,
  scale= 30,
  maxPixels= 1e9
)

class2 = meanDictionary2.getInfo()

import pandas as pd
df =pd.DataFrame(list(class2.items()),columns=['Band','pixels'])
# extracting the number of pixels from the valueinfo27
acreage2=(df.iloc[0]['pixels'])*30*30*0.000001
print('sq km Pollen: ',acreage2) #acreage of pixels in sq kilometers

sq km Pollen:  4.632914117647061


In [33]:
#calculating areage 

# THIS CODE CREATES A BINARY IMAGE OF YOUR INDEX, POSITIVE VALUES ARE 1 NEGATIVE VALUES 0
binary_im =rfClassificationImage2015.eq(0) #class 0 is water, 
Map.addLayer(binary_im)

# // Reduce the region. The region parameter is the Feature geometry. THIS IS THE CODE FOR ADDING THE BINARY IMAGE
meanDictionary0 = binary_im.reduceRegion(
  reducer= ee.Reducer.sum(), #sum of ones from the binary map
  geometry= region,
  scale= 30,
  maxPixels= 1e9
)

class0 = meanDictionary0.getInfo()

import pandas as pd
df =pd.DataFrame(list(class0.items()),columns=['Band','pixels'])
# extracting the number of pixels from the valueinfo27
acreage0=(df.iloc[0]['pixels'])*30*30*0.000001
print('sq km Water: ',acreage0) #acreage of pixels in sq kilometers

sq km Water:  47.9771752941176


In [34]:
#calculating areage 

# THIS CODE CREATES A BINARY IMAGE OF YOUR INDEX, POSITIVE VALUES ARE 1 NEGATIVE VALUES 0
binary_im =rfClassificationImage2015.eq(1) #class 1 is land, 
Map.addLayer(binary_im)

# // Reduce the region. The region parameter is the Feature geometry. THIS IS THE CODE FOR ADDING THE BINARY IMAGE
meanDictionary1 = binary_im.reduceRegion(
  reducer= ee.Reducer.sum(), #sum of ones from the binary map
  geometry= region,
  scale= 30,
  maxPixels= 1e9
)

class1 = meanDictionary1.getInfo()

import pandas as pd
df =pd.DataFrame(list(class1.items()),columns=['Band','pixels'])
# extracting the number of pixels from the valueinfo27
acreage1=(df.iloc[0]['pixels'])*30*30*0.000001
print('sq Km Island: ',acreage1) #acreage of pixels in sq kilometers

sq Km Island:  2.0941235294117613


In [35]:
#################### 2018 Area ###############################

In [36]:
#calculating areage 

# THIS CODE CREATES A BINARY IMAGE OF YOUR INDEX, POSITIVE VALUES ARE 1 NEGATIVE VALUES 0
binary_im =rfClassificationImage2018.eq(2) #class 2 is pollen, any other class is ignored
#Map.addLayer(binary_im)

# // Reduce the region. The region parameter is the Feature geometry. THIS IS THE CODE FOR ADDING THE BINARY IMAGE
meanDictionary2 = binary_im.reduceRegion(
  reducer= ee.Reducer.sum(), #sum of ones from the binary map
  geometry= region,
  scale= 30,
  maxPixels= 1e9
)

class2 = meanDictionary2.getInfo()

import pandas as pd
df =pd.DataFrame(list(class2.items()),columns=['Band','pixels'])
# extracting the number of pixels from the valueinfo27
acreage2=(df.iloc[0]['pixels'])*30*30*0.000001
print('sq km Pollen: ',acreage2) #acreage of pixels in sq kilometers

sq km Pollen:  11.867636470588247


In [37]:
#calculating areage 

# THIS CODE CREATES A BINARY IMAGE OF YOUR INDEX, POSITIVE VALUES ARE 1 NEGATIVE VALUES 0
binary_im =rfClassificationImage2018.eq(0) #class 0 is water, 
Map.addLayer(binary_im)

# // Reduce the region. The region parameter is the Feature geometry. THIS IS THE CODE FOR ADDING THE BINARY IMAGE
meanDictionary0 = binary_im.reduceRegion(
  reducer= ee.Reducer.sum(), #sum of ones from the binary map
  geometry= region,
  scale= 30,
  maxPixels= 1e9
)

class0 = meanDictionary0.getInfo()

import pandas as pd
df =pd.DataFrame(list(class0.items()),columns=['Band','pixels'])
# extracting the number of pixels from the valueinfo27
acreage0=(df.iloc[0]['pixels'])*30*30*0.000001
print('sq km Water: ',acreage0) #acreage of pixels in sq kilometers

sq km Water:  40.5751376470588


In [38]:
#calculating areage 

# THIS CODE CREATES A BINARY IMAGE OF YOUR INDEX, POSITIVE VALUES ARE 1 NEGATIVE VALUES 0
binary_im =rfClassificationImage2018.eq(1) #class 1 is land, 
Map.addLayer(binary_im)

# // Reduce the region. The region parameter is the Feature geometry. THIS IS THE CODE FOR ADDING THE BINARY IMAGE
meanDictionary1 = binary_im.reduceRegion(
  reducer= ee.Reducer.sum(), #sum of ones from the binary map
  geometry= region,
  scale= 30,
  maxPixels= 1e9
)

class1 = meanDictionary1.getInfo()

import pandas as pd
df =pd.DataFrame(list(class1.items()),columns=['Band','pixels'])
# extracting the number of pixels from the valueinfo27
acreage1=(df.iloc[0]['pixels'])*30*30*0.000001
print('sq Km Island: ',acreage1) #acreage of pixels in sq kilometers

sq Km Island:  2.2614388235294087


In [39]:
#################### 2019 Area ###############################

In [40]:
#calculating areage 

# THIS CODE CREATES A BINARY IMAGE OF YOUR INDEX, POSITIVE VALUES ARE 1 NEGATIVE VALUES 0
binary_im =rfClassificationImage2019.eq(2) #class 2 is pollen, any other class is ignored
#Map.addLayer(binary_im)

# // Reduce the region. The region parameter is the Feature geometry. THIS IS THE CODE FOR ADDING THE BINARY IMAGE
meanDictionary2 = binary_im.reduceRegion(
  reducer= ee.Reducer.sum(), #sum of ones from the binary map
  geometry= region,
  scale= 30,
  maxPixels= 1e9
)

class2 = meanDictionary2.getInfo()

import pandas as pd
df =pd.DataFrame(list(class2.items()),columns=['Band','pixels'])
# extracting the number of pixels from the valueinfo27
acreage2=(df.iloc[0]['pixels'])*30*30*0.000001
print('sq km Pollen: ',acreage2) #acreage of pixels in sq kilometers

sq km Pollen:  8.755235294117652


In [41]:
#calculating areage 

# THIS CODE CREATES A BINARY IMAGE OF YOUR INDEX, POSITIVE VALUES ARE 1 NEGATIVE VALUES 0
binary_im =rfClassificationImage2019.eq(0) #class 0 is water, 
Map.addLayer(binary_im)

# // Reduce the region. The region parameter is the Feature geometry. THIS IS THE CODE FOR ADDING THE BINARY IMAGE
meanDictionary0 = binary_im.reduceRegion(
  reducer= ee.Reducer.sum(), #sum of ones from the binary map
  geometry= region,
  scale= 30,
  maxPixels= 1e9
)

class0 = meanDictionary0.getInfo()

import pandas as pd
df =pd.DataFrame(list(class0.items()),columns=['Band','pixels'])
# extracting the number of pixels from the valueinfo27
acreage0=(df.iloc[0]['pixels'])*30*30*0.000001
print('sq km Water: ',acreage0) #acreage of pixels in sq kilometers

sq km Water:  43.70762117647058


In [42]:
#calculating areage 

# THIS CODE CREATES A BINARY IMAGE OF YOUR INDEX, POSITIVE VALUES ARE 1 NEGATIVE VALUES 0
binary_im =rfClassificationImage2019.eq(1) #class 1 is land, 
Map.addLayer(binary_im)

# // Reduce the region. The region parameter is the Feature geometry. THIS IS THE CODE FOR ADDING THE BINARY IMAGE
meanDictionary1 = binary_im.reduceRegion(
  reducer= ee.Reducer.sum(), #sum of ones from the binary map
  geometry= region,
  scale= 30,
  maxPixels= 1e9
)

class1 = meanDictionary1.getInfo()

import pandas as pd
df =pd.DataFrame(list(class1.items()),columns=['Band','pixels'])
# extracting the number of pixels from the valueinfo27
acreage1=(df.iloc[0]['pixels'])*30*30*0.000001
print('sq Km Island: ',acreage1) #acreage of pixels in sq kilometers

sq Km Island:  2.2413564705882325
